In [1]:
import csv

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [68]:
train_data = pd.read_csv('train.csv')

In [53]:
train_data = train_data.fillna(0)

In [69]:
train_data.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        8
Discontinued        0
dtype: int64

In [71]:
train_data.shape

(5343, 21)

In [ ]:
train_data

In [54]:
non_categorical_cols = train_data.select_dtypes(include=['int64', 'float64']).columns

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data[non_categorical_cols] = scaler.fit_transform(train_data[non_categorical_cols])

In [55]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = train_data.select_dtypes(include=['object', 'category']).columns

label_encoder = LabelEncoder()
for col in categorical_cols:
    train_data[col] = label_encoder.fit_transform(train_data[col].astype(str))

In [70]:
train_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Discontinued
0,1915-IOFGU,Female,0,No,No,1,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,No,Electronic check,70.50,70.50,Yes
1,6728-CZFEI,Female,0,No,No,15,Yes,No,DSL,No,...,No,No,Yes,No,One year,No,Mailed check,56.15,931.90,No
2,3863-IUBJR,Male,0,Yes,Yes,12,Yes,No,DSL,No,...,No,No,No,Yes,One year,No,Credit card (automatic),53.65,696.35,Yes
3,5572-ZDXHY,Female,0,No,No,22,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,Yes,Mailed check,84.30,1855.65,Yes
4,8348-HFYIV,Male,0,No,No,2,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,49.25,90.35,Yes


In [57]:
# train a random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X = train_data.drop(['customerID', 'Discontinued'], axis=1)
y = train_data['Discontinued']

In [30]:
random_forest_regressor = RandomForestRegressor(n_estimators=100, random_state=1)  
random_forest_regressor.fit(X, y)

RandomForestRegressor(random_state=42)

In [59]:
test_data = pd.read_csv('test.csv')
non_categorical_cols = test_data.select_dtypes(include=['int64', 'float64']).columns

test_data = test_data.fillna(0)
test_data[non_categorical_cols] = scaler.fit_transform(test_data[non_categorical_cols])

categorical_cols = test_data.select_dtypes(include=['object', 'category']).columns

for col in categorical_cols:
    test_data[col] = label_encoder.fit_transform(test_data[col].astype(str))

test_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,55,0,-0.439298,1,1,-0.609724,1,2,1,0,0,2,0,2,0,0,0,2,0.746442,-0.365500
1,806,1,-0.439298,0,0,-1.258848,1,0,2,1,1,1,1,1,1,0,0,3,-1.500623,-0.993690
2,874,1,2.276361,0,0,0.242252,1,2,1,0,0,0,0,2,2,0,0,2,0.962028,0.590796
3,597,1,-0.439298,0,0,-1.218278,1,2,1,0,2,0,0,0,0,0,1,2,0.454572,-0.939318
4,972,0,-0.439298,0,0,0.607385,1,2,1,0,2,2,0,2,2,0,1,2,1.346765,1.112532


In [60]:
test_data.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64

In [46]:
test_data = test_data.fillna(0)

In [61]:
X = test_data.drop(['customerID'], axis=1)
y = random_forest_regressor.predict(X)

In [63]:
pred = pd.DataFrame(y, columns=['Discontinued'])
test_data = pd.read_csv('test.csv')
pred['customerID'] = test_data['customerID']
# switch columns
pred = pred[['customerID', 'Discontinued']]
# change customerID to ID
pred.rename(columns={'customerID': 'ID'}, inplace=True)
pred.to_csv('submission.csv', index=False)

In [50]:
pred.shape

(1700, 2)

In [65]:
pred.describe()

,Discontinued
count,1700.000000
mean,0.370518
std,0.169379
min,0.070000
25%,0.220000
50%,0.380000
75%,0.510000
max,0.730000
